# Language Detection Model
This notebook implements a language detection pipeline, with improvements in preprocessing, model training, and evaluation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Load the dataset
file_path = '/content/drive/My Drive/Colab Notebooks/language.csv'
df = pd.read_csv(file_path)

# Basic exploration
print(df.head())
print(df.isnull().sum())
print(df['language'].value_counts())

In [ ]:
# Visualize class distribution
plt.figure(figsize=(8, 8))
df['language'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Language Distribution')
plt.xlabel('Language')
plt.ylabel('Count')
plt.show()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Preprocessing function
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing
df['cleaned_text'] = df['Text'].apply(preprocess_text)
print(df['cleaned_text'].head())

In [ ]:
# Split the dataset
X = df['cleaned_text']
y = df['language']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f'Training samples: {len(X_train)}')
print(f'Testing samples: {len(X_test)}')

In [ ]:
# Vectorize text data using TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print('TF-IDF feature matrix shape:', X_train_tfidf.shape)

In [ ]:
# Train a Logistic Regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model.predict(X_test_tfidf)

print('Classification Report:')
print(classification_report(y_test, y_pred))

# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=model.classes_, yticklabels=model.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Predict user input
user_input = input('Enter a text: ')
user_tfidf = tfidf.transform([user_input])
prediction = model.predict(user_tfidf)
print(f'Predicted Language: {prediction[0]}')